In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
os.chdir("/kaggle/input/")
# os.mkdir('/kaggle/input/slim_trained')
for dirname, _, filenames in os.walk('../input/siim-covid19-detectionut'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
data_image = pd.read_csv(r'../input/siim-covid19-detection/train_image_level.csv')
data_study  = pd.read_csv(r'../input/siim-covid19-detection/train_study_level.csv')

# Concatinate the above files by ID's

Step1: Get the Path for the images

In [3]:
trainimlist = []

for dirname, _, filenames in os.walk('../input/siim-covid19-detection/train'):
    for filename in filenames:
        trainimlist.append(os.path.join(dirname, filename))

Step2: check image_id and train/Id's are the same

In [4]:
# check id in train_imgage_level and Names in the train folder are the same

# to store path with order by id in train_imgage_level

Path = [] 

for image_id in data_image['id']:
    target = image_id.replace('_image', '')
    for path in trainimlist:
        if target in path: 
            Path.append(path)

Adding the Path list to the data_image file

In [5]:
data_image["Path"] = pd.Series(Path, index=data_image.index)

# data_image id (6334) and images in train folder (6334) are one-to-one relationships
len(data_image['Path'].unique().tolist())

6334

In [6]:
# checking 
data_image.head(3)

,id,boxes,label,StudyInstanceUID,Path
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,../input/siim-covid19-detection/train/5776db0c...
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed,../input/siim-covid19-detection/train/ff0879eb...
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,../input/siim-covid19-detection/train/9d514ce4...


In [7]:
for itr in data_study['id']:
    data_study['id'] = data_study['id'].str.replace('_study', '')

StudyInstanceID and data_study['id'] are related.
So, removing _study from data_study for merge the 2 data-frames.

In [8]:
data_study = data_study.rename(columns={'id': "StudyInstanceUID"})

This step is for merge as merge requires the same column name

In [9]:
train = pd.merge(data_image, data_study, on='StudyInstanceUID')

Only rows are kept for which common keys are found in both data frames.
In case you want to keep all rows from the left data frame (data_image) and only add values from data_study where a matching key is available, you can use how="left".

In [10]:
# saving data_frame: check output /kaggle/working/slim folder 
train.to_csv('/kaggle/working/train_data.csv',index=False)